In [ ]:
import random
from llama_index.core.workflow import Context
from llama_index.llms.openai import OpenAI
from openai import OpenAI as OpenAI_from_openai
from dummy_data import user_information, product_information, product_reviews
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent, AgentWorkflow
from llama_index.utils.workflow import draw_all_possible_flows
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
import json
from llama_index.core.tools import FunctionTool
import time
import asyncio
from datetime import datetime
from dummy_data import user_information, product_information, product_reviews

# print("----------------------------------------------------------------------------")
# print("User Information: ", user_information)
# print("----------------------------------------------------------------------------")
# print("Product Information: ", product_information)
# print("----------------------------------------------------------------------------")
# print("Product Reviews: ", product_reviews)

# Initialize OpenAI LLM
llm = OpenAI(model="gpt-4o", api_key=" ")

data = {}

async def retrieve_user_profile() -> str:
    """Fetches user information."""
    print(f"\n$$$$$$$   [{datetime.now()}] START OF USER_PROFILE   $$$$$$$\n")

    global data

    await asyncio.sleep(3)
    data = data
    users = user_information
    user_id_to_return = 3
    data["user_profile"] = users[str(user_id_to_return)]

    print(f"\n$$$$$$$   [{datetime.now()}] END OF USER_PROFILE   $$$$$$$\n")

    return "User profile fetched."

async def retrieve_product_information() -> str:
    """Fetches product information."""
    print(f"\n$$$$$$$   [{datetime.now()}] START OF PRODUCT_INFORMATION   $$$$$$$\n")

    global data

    await asyncio.sleep(3)
    data = data
    data["product_information"] = product_information

    print(f"\n$$$$$$$   [{datetime.now()}]  END OF PRODUCT_INFORMATION   $$$$$$$\n")

    return "Product information fetched."

async def retrieve_product_reviews() -> str:
    """Fetches product reviews."""
    
    global data

    print(f"\n$$$$$$$   [{datetime.now()}]   START OF PRODUCT_REVIEWS   $$$$$$$\n")

    await asyncio.sleep(3)
    data = data
    data["product_reviews"] = product_reviews

    print(f"\n$$$$$$$   [{datetime.now()}]   END OF PRODUCT_REVIEWS   $$$$$$$\n")


    return "Product reviews fetched."


async def features_highlighting() -> str:
    """Analyzes user profile and product data to highlight relevant features of the product based on the user profile."""
    print(f"\n$$$$$$$   [{datetime.now()}]   START OF FEATURES_HIGHLIGHTING   $$$$$$$\n")

    global data

    await asyncio.sleep(3)
    data = data
    user_profile = data.get("user_profile", {})
    product_info = data.get("product_information", {})

    prompt_template = f"""
    You are given a user profile and product information. Analyze the details of both and suggest which features of the product will be most appealing to the user. As your final output, return only a list of features. Do not put a variable name before the list. Your response should only contain the list and nothing else:
    
    [
        **List of featrures selected**
    ]
    
    Perform the task for the following information:
    User Profile: {user_profile}
    Product Information: {product_info}
    """
    
    client = OpenAI_from_openai(api_key=" ")
    messages = [{"role": "user", "content": prompt_template}]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0.1,
        top_p=0.1,
        messages=messages
    ).choices[0].message.content.strip()

    data["suggested_features"] = response

    print(f"\n$$$$$$$   [{datetime.now()}]   END OF FEATURES_HIGHLIGHTING   $$$$$$$\n")


    return "Features suggested."

async def sort_reviews() -> str:
    """Analyzes user profile and the list of reviews and sort the reviews according to the user profile. The reviews that are most likely to be of interest to the user should be coming first in the list of reviews."""
    print(f"\n$$$$$$$   [{datetime.now()}]   START OF SORT REVIEWS   $$$$$$$\n")

    global data

    await asyncio.sleep(3)
    data = data
    user_profile = data.get("user_profile", {})
    product_reviews = data.get("product_reviews", [])

    prompt_template = f"""
    You are given a user profile and a list of product reviews. Analyze the details of both and sort the reviews based on the user profile information. The reviews that match the users profile the most and are the ones that the user will most likely be interested in should come first. As your final output, return only a list of reviews sorted based on the user profile information. Do not put a variable name before the list. Your response should only contain the list and nothing else:
    
    [
        **List of reviews sorted**
    ]

    User Profile: {user_profile}
    Product Reviews: {product_reviews}
    """
    
    client = OpenAI_from_openai(api_key=" ")
    messages = [{"role": "user", "content": prompt_template}]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0.1,
        top_p=0.1,
        messages=messages
    ).choices[0].message.content.strip()

    data["sorted_reviews"] = response

    print(f"\n$$$$$$$   [{datetime.now()}]   END OF SORT_REVIEWS   $$$$$$$\n")


    return "reviews sorted."

async def custom_description() -> str:
    """Analyzes user profile and product data to create a custom product description that will be most intriguing to the user."""
    print(f"\n$$$$$$$   [{datetime.now()}]  START OF CUSTOM_DESCRIPTION   $$$$$$$\n")

    global data

    await asyncio.sleep(3)
    data = data
    user_profile = data.get("user_profile", {})
    product_info = data.get("product_information", {})

    prompt_template = f"""
    You are given a user profile and product information. You have to create a custom description of the product based on the user profile information. The description should be most appealing to the user and should not be longer than 1 sentence.  As your final output, return a json object ONLY, like the follwowing. Do not put a variable name before the json object. Your response should only contain the json object and nothing else:
    {{
        "custom_description": **The custom description that you create**
    }}
    
    User Profile: {user_profile}
    Product Information: {product_info}
    """
    
    client = OpenAI_from_openai(api_key=" ")
    messages = [{"role": "user", "content": prompt_template}]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0.1,
        top_p=0.1,
        messages=messages
    ).choices[0].message.content.strip()

    data["custom_description"] = response

    print(f"\n$$$$$$$   [{datetime.now()}]   END OF CUSTOM_DESCRIPTION   $$$$$$$\n")
    return "created custom description."


# converting functions to tools
retrieve_user_profile_tool = FunctionTool.from_defaults(retrieve_user_profile)
retrieve_product_information_tool = FunctionTool.from_defaults(retrieve_product_information)
retrieve_product_reviews_tool = FunctionTool.from_defaults(retrieve_product_reviews)
features_highlighting_tool = FunctionTool.from_defaults(features_highlighting)
sort_reviews_tool = FunctionTool.from_defaults(sort_reviews)
custom_description_tool = FunctionTool.from_defaults(custom_description)

# Define Agents
from llama_index.core.agent import FunctionCallingAgent
data_agent = FunctionCallingAgent.from_tools(
    system_prompt="""You are an agent who has tools to fetch data. You have the following tools: 
    - retrieve_user_profile: this tool is used to fetch user information. Use this when you have to fetch the user details.
    - retrieve_product_information: this tool is used to fetch product information. Use this when you have to fetch the product details.
    - retrieve_product_reviews: this tool is used to fetch product reviews. Use this when you have to fetch the product reviews.
    All of these tools perform tasks independently. You must always call them in parallel.
    """,
    llm=llm,
    tools=[retrieve_user_profile_tool, retrieve_product_information_tool, retrieve_product_reviews_tool],
    allow_parallel_tool_calls=True,
    verbose=False
)

customization_agent = FunctionCallingAgent.from_tools(
    system_prompt="""You are an agent that is used to make customizations based on the user profile. You have the following tools.
    - features_highlighting: this tool is used to suggest which features should be highlighted. Based on the users profile, this tool will suggest which features of the product should be highlighted. Call this tool for features_highlighting.
    - sort_reviews: this tool is used to sort the reviews based on the user profile. Call this tool to sort the reviews based on the user profile.
    - custom_description: this tool is used to create a custom product description based on the user profile. Call this tool to create a custom product description.
    
    These tools are used for customization based on the user profile. Use the 'features_highlighting' tool to suggest which features should be highlighted. Use the 'sort_reviews' tool to sort the reviews based on the user profile. Use the 'custom_description' tool to create a custom product description based on the user profile.

    All of these tools perform tasks independently. You must always call them in parallel.

    """,
    llm=llm,
    tools=[features_highlighting_tool, sort_reviews_tool, custom_description_tool],
    allow_parallel_tool_calls=True,
    verbose=False
)


# running the agents
async def run_agents():

    start = time.time()
    result_data_agent = await data_agent.achat("Gather product details, user details and product reviews")
    print("Data Agent Result: ", result_data_agent)
    end = time.time()
    print("Time taken by Data Agent: ", end-start)


    # if result_data_agent:
    #     result_customization_agent = await customization_agent.achat("Customize the product based on user profile. Suggest features to be highlighted, sort reviews and create a custom description.")
    #     print("Customization Agent Result: ", result_customization_agent)

import nest_asyncio
nest_asyncio.apply()

# Run the agents
start = time.time()
await run_agents()
end = time.time()


$$$$$$$   [2025-02-18 20:15:55.059677] START OF USER_PROFILE   $$$$$$$


$$$$$$$   [2025-02-18 20:15:55.059677] START OF PRODUCT_INFORMATION   $$$$$$$


$$$$$$$   [2025-02-18 20:15:55.064756]   START OF PRODUCT_REVIEWS   $$$$$$$


$$$$$$$   [2025-02-18 20:15:58.077215] END OF USER_PROFILE   $$$$$$$


$$$$$$$   [2025-02-18 20:15:58.077215]  END OF PRODUCT_INFORMATION   $$$$$$$


$$$$$$$   [2025-02-18 20:15:58.078274]   END OF PRODUCT_REVIEWS   $$$$$$$

Data Agent Result:  I have successfully gathered the following information:

1. **User Details**: User profile fetched.
2. **Product Details**: Product information fetched.
3. **Product Reviews**: Product reviews fetched.
